In [ ]:
import pandas as pd
import yfinance as yf
import ta
import plotly.graph_objs as go
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import psycopg2
import zlib
import pickle as pk
from dotenv import load_dotenv
import os 
from sklearn.model_selection import train_test_split
from pandas.tseries.offsets import BDay


In [ ]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]

sp500["Symbol"] = sp500["Symbol"].str.replace('.', "-")

In [ ]:
sp500

In [ ]:
symbol_list = sp500["Symbol"].unique().tolist()

In [ ]:
end_date = "2024-06-24"

start_date = "2015-01-30"

df = yf.download(tickers=tickers,
                 start=start_date,
                 end=end_date).stack()
df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df = df.reorder_levels(['ticker', 'date'])

df = df.sort_index()



In [ ]:
df.to_csv("stock.csv")

In [ ]:
stocks = pd.read_csv("Data/stock.csv")
stocks.tail()

In [ ]:
stocks

In [ ]:
tickers = stocks['ticker'].unique().tolist()
tickers = sorted(tickers)

In [ ]:
tickers

In [ ]:
multi_index_df = df

In [ ]:
df = multi_index_df.copy()

In [ ]:
def process_df(df:pd.DataFrame, tickers:list):
    processed_df = pd.DataFrame()

    for tick in tickers:
        ticker_data = df.loc[tick].copy()
        
        price = (ticker_data['high'] + ticker_data['low'] + ticker_data['adj close']) / 3
        pv = ticker_data['volume'] * price
        cumu_pv = pv.cumsum()
        cumu_vol = ticker_data['volume'].cumsum()
        ticker_data["vwap"] = cumu_pv / cumu_vol
        
        # Calculate RVI
        StdDev_high = ticker_data['high'].rolling(window=14).std()
        StdDev_low = ticker_data['low'].rolling(window=14).std()
        ticker_data['RVI'] = 100 * (StdDev_high / StdDev_high + StdDev_low)
        ticker_data["RVI_Signal"] = ticker_data['RVI'].rolling(window=4).mean()

        # Calculate MFI
        typical_price = (ticker_data['high'] + ticker_data['low'] + ticker_data['adj close']) / 3
        raw_money_flow = typical_price * ticker_data['volume']
        positive_money_flow = raw_money_flow.where(typical_price > typical_price.shift(1), 0)
        negative_money_flow = raw_money_flow.where(typical_price < typical_price.shift(1), 0)
        positive_money_flow_sum = positive_money_flow.rolling(window=14).sum()
        negative_money_flow_sum = negative_money_flow.rolling(window=14).sum()
        money_flow_ratio = positive_money_flow_sum / negative_money_flow_sum
        ticker_data['MFI'] = 100 - (100 / (1 + money_flow_ratio))
        
        # Calculate On Balance Volume
        ticker_data['OBV'] = (ticker_data['adj close'].diff() > 0) * ticker_data['volume'] - (ticker_data['adj close'].diff() < 0) * ticker_data['volume']
        ticker_data['OBV'] = ticker_data['OBV'].fillna(0).cumsum()

        # Calculate Relative Strength Index
        ticker_data['rsi'] = ta.momentum.rsi(ticker_data['adj close'], window=14)

        # Calculate Bollinger Bands
        bollinger = ta.volatility.BollingerBands(close=ticker_data['adj close'], window=20, window_dev=2)
        ticker_data['bb_low'] = bollinger.bollinger_lband()
        ticker_data['bb_mid'] = bollinger.bollinger_mavg()
        ticker_data['bb_high'] = bollinger.bollinger_hband()

        # Calculate Average True Range
        ticker_data['atr'] = ta.volatility.average_true_range(ticker_data['high'], ticker_data['low'], ticker_data['adj close'], window=14)

        # Calculate Moving Average Convergence Divergence
        macd = ta.trend.MACD(ticker_data['close'])
        ticker_data['macd'] = macd.macd()
        ticker_data['macd_signal'] = macd.macd_signal()
        
        # Calculate Moving Averages
        ticker_data['short_ma'] = ta.trend.sma_indicator(ticker_data['adj close'], window=20)  
        ticker_data['long_ma'] = ta.trend.sma_indicator(ticker_data['adj close'], window=50)
        
        
        ticker_data['rolling_mean'] = ticker_data['adj close'].rolling(window=20).mean()
        ticker_data['rolling_std'] = ticker_data['adj close'].rolling(window=20).std()
        
        ticker_data['return'] = ticker_data['adj close'].pct_change()
        
        # Remove NaN values
        ticker_data = ticker_data.dropna(subset=[i for i in ticker_data.columns])

        ticker_data['ticker'] = tick
        ticker_data = ticker_data.set_index('ticker', append=True)
        ticker_data = ticker_data.reorder_levels(['ticker', 'date'])

        processed_df = pd.concat([processed_df, ticker_data])
        print(tick)
        
    return processed_df
    
    



In [ ]:
processed_df = process_df(df)

In [ ]:
load_dotenv()
db_params = {
    'host': os.getenv('SERVER'),
    'dbname': os.getenv("DATABASE"),
    'user': os.getenv("USERNAME"),
    'password': os.getenv("PASSWORD"),
    'port': os.getenv("PORT_ID")
}
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

In [ ]:
def create_table_if_not_exists(db_params):
    """Create the models table if it doesn't exist."""
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS models (
            ticker VARCHAR PRIMARY KEY UNIQUE,
            all_feature_model BYTEA,
            close_price_model BYTEA,
            high_price_model BYTEA,
            low_price_model BYTEA,
            open_price_model BYTEA,
            volume_model BYTEA
        );
    """)
    print("created table")
    conn.commit()
    cursor.close()
    conn.close()

In [ ]:
def store_models(model,tick, column_name, db_params):

    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    
    model_data = zlib.compress(pk.dumps(model))
    cursor.execute(f"""
        INSERT INTO models (ticker, {column_name}) 
        VALUES (%s, %s)
        ON CONFLICT (ticker) DO UPDATE 
        SET {column_name} = EXCLUDED.{column_name};
    """, (tick, model_data))
    print(f"Stored model for {tick}")
    conn.commit()
    cursor.close()
    conn.close()


In [ ]:
def get_model(db_params, tick, column_name):
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    cursor.execute(f"SELECT {column_name} FROM models WHERE ticker = %s", (tick,))
    result = cursor.fetchone()

    if result and result[0]:
        compressed_model = result[0]
        model_data = zlib.decompress(compressed_model)
        model = pk.loads(model_data)
        return model
    else:
        raise ValueError(f"No model found for {tick}")

In [ ]:
def train_all_to_DB(df, tickers):
    # models_all = {}

    count = 0
    for tick in tickers:
        stock_data = df.loc[tick].copy() 

        
        X = stock_data[[    
                    "close",
                    "high",
                    "low",
                    "open",
                    "volume",
                    "vwap",
                    "rsi",
                    "bb_low",
                    "bb_mid",
                    "bb_high",
                    "atr",
                    "macd",
                    "macd_signal",
                    "short_ma",
                    "long_ma",
                    "return",
                    "rolling_mean",
                    "rolling_std",
                    "lag1",
                    "lag2",
                    "lag3",
                    "target"]]
        y = stock_data['adj close']

        # Split into training and testing sets
        train_data = stock_data.loc[stock_data.index]
        
        X_train = train_data.drop(columns=['adj close'])
        y_train = train_data['adj close']
        
        # Model Training
        model_all = RandomForestRegressor(n_estimators=100, random_state=42)
        
        print(f'Shape: {X_train.shape}')
        model_all.fit(X_train, y_train)
        store_models(model_all, tick, 'all_feature_model', db_params)
        # models_all[tick] = model_all
        
        count +=1 
        print(f'{count}: {tick}')
    

In [ ]:
def single_feature_train(data: pd.DataFrame,feature: str, test_size, column_name): 
    feature_data = data[feature].to_frame(feature)
    mean_sqr_error = {}
    feature_models = {}
    pred = {}
    actual = {}
    dates = {}
    if test_size == 0:
        count = 0
        for tick in tickers:
            y = feature_data.loc[tick][feature]
            X = pd.DataFrame(index=y.index)
            X[f'{feature}_1'] = y.shift(1)
            X[f'{feature}_2'] = y.shift(2)
            X[f'{feature}_3'] = y.shift(3)
            X = X.dropna()
            y = y.loc[X.index]
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X, y)
            store_models(model, tick, column_name, db_params)
            count +=1
            print(f"{count}: {tick}")
        return "Finished saving all models to database"
    else:
        count = 0
        for tick in tickers:    
            y = feature_data.loc[tick][feature]

            # Create features using previous days' closing prices
            X = pd.DataFrame(index=y.index)
            X[f'{feature}_1'] = y.shift(1)
            X[f'{feature}_2'] = y.shift(2)
            X[f'{feature}_3'] = y.shift(3)

            X = X.dropna()
            y = y.loc[X.index]

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

            model = RandomForestRegressor(n_estimators=100, random_state=42)

            model.fit(X_train, y_train)
            feature_models[tick] = model
            
            y_pred = model.predict(X_test)

            pred[tick] = y_pred
            
            mse = mean_squared_error(y_test, y_pred)
            mean_sqr_error[tick] = mse
            actual[tick] = y_test.values
            dates = X_test.index
            count +=1
            print(f"{count}: {tick} - Mean Squared Error: {mse}")
        return feature_models, pred, actual, dates

In [ ]:
column_list = ["close_price_model","high_price_model" ,"low_price_model" ,"open_price_model", "volume_model"]

In [ ]:
features = ["close","high","low","open","volume"]

In [ ]:
def predict_single_feature_model(data, tickers, features, column_names, db_params, prediction_days=5):
    index = pd.MultiIndex(levels=[[], []], codes=[[], []], names=['ticker', 'date'])
    predictions_df = pd.DataFrame(index=index, columns=features)

    for feature, column_name in zip(features, column_names):
        feature_data = data[feature].to_frame(feature)

        for tick in tickers:
            y = feature_data.loc[tick][feature]
            X = pd.DataFrame(index=y.index)
            X[f'{feature}_1'] = y.shift(1)
            X[f'{feature}_2'] = y.shift(2)
            X[f'{feature}_3'] = y.shift(3)
            X = X.dropna()
            last_features = X.iloc[-1:]
            model = get_model(db_params, tick, column_name)

            # Predict for the required number of days
            for days in range(prediction_days):
                prediction = model.predict(last_features)[0]

                last_date = pd.to_datetime(last_features.index[0])
                next_date = last_date + BDay(1)  # Ensure it's a business day
                next_date = pd.to_datetime(next_date)

                # Add prediction to the DataFrame
                predictions_df.loc[(tick, next_date.strftime('%Y-%m-%d')), feature] = prediction

                # Update the last_features dataframe for the next prediction
                new_row = {
                    f'{feature}_1': last_features[f'{feature}_2'].iloc[-1],
                    f'{feature}_2': last_features[f'{feature}_3'].iloc[-1],
                    f'{feature}_3': prediction
                }
                new_last_features = pd.DataFrame(new_row, index=[next_date.strftime('%Y-%m-%d')])
                last_features = pd.concat([last_features, new_last_features])
                last_features = last_features.tail(1)  # Keep only the last row
                print(f"{next_date.strftime('%Y-%m-%d')} -{tick} - {feature}: {prediction}")

    return predictions_df

In [ ]:
pred_valuess = predict_single_feature_model(df,tickers,features,column_list, db_params, 7)

In [ ]:
adj_close_pred_df = pd.DataFrame()

for tick in tickers:
    predictions = get_model(db_params, tick, "all_feature_model").predict(pred_values_df.loc[tick])
    dates = pred_values_df.loc[tick].index
    temp_df = pd.DataFrame({'ticker': tick, 'date': dates, 'adj close future': predictions})
    temp_df.set_index(['ticker', 'date'], inplace=True)
    adj_close_pred_df = pd.concat([adj_close_pred_df, temp_df])
    print(f"Predictions for  {tick}:\n {predictions}\n")

In [ ]:
end_date = "2024-07-04"

start_date = "2024-06-25"

df_new = yf.download(tickers=tickers, 
                     start=start_date, 
                     end=end_date).stack()

# Set the index names
df_new.index.names = ['date', 'ticker']

df_new.columns = df_new.columns.str.lower()

df_new = df_new.reorder_levels(['ticker', 'date'])

df_new = df_new.sort_index()

In [ ]:
def mse_future(adj_close_future, adj_close_pred_df, tickers):
    
    results = []

    
    for ticker in tickers:
        
        future_values = adj_close_future.loc[ticker]
        pred_values = adj_close_pred_df.loc[ticker]
        
        
        for date in future_values.index:
            if date in pred_values.index:
                
                mse = mean_squared_error([future_values.loc[date]], [pred_values.loc[date]])
                
                results.append((ticker, date, mse, future_values.loc[date], pred_values.loc[date][0]))
    
    
    mse_df = pd.DataFrame(results, columns=['ticker', 'date', 'mse', 'adj_close_future', 'adj_close_pred'])
    
    mse_df.set_index(['ticker', 'date'], inplace=True)
    
    return mse_df

In [ ]:
mse_df = mse_future(adj_close_future, adj_close_pred_df, tickers)

In [ ]:


# def plot_market(data, ticker:list, market:str):
#     fig = go.Figure()
#     for tick in ticker:
#         if tick not in tickers:
#             return f'{tick} not valid ticker'
#     if market == 'volume':
#         yaxis = 'Volume' 
#     elif market in data.columns:
#         yaxis = f'{market.capitalize()} Prince'
#     else:
#         return 'Enter valid Market'
        
#     for tick in ticker:
#         fig.add_trace(go.Scatter(
#             x=data.loc[tick].index,
#             y=data.loc[tick][market],
#             mode='lines',
#             name=tick
#         ))
#     fig.update_layout(
#         title=f'Stock {market} for Multiple Ticker/s',
#         xaxis_title='Date',            
#         yaxis_title=yaxis,
#         legend_title='Ticker'
#     )
#     fig.show()
    




In [ ]:
# import dash
# from dash import dcc, html
# from dash.dependencies import Input, Output
# import plotly.graph_objs as go



# app = dash.Dash(__name__)

# app.layout = html.Div([
#     dcc.Dropdown(
#         id='stock-selector',
#         options=[{'label': ticker, 'value': ticker} for ticker in tickers],
#         value=df.index.get_level_values(0).unique()[0]
#     ),
#     dcc.Graph(id='price-chart')
# ])

# @app.callback(
#     Output('price-chart', 'figure'),
#     [Input('stock-selector', 'value')]
# )
# def update_chart(selected_stock):
#     ticker_df = df.loc[selected_stock]
#     figure = {
#         'data': [
#             go.Scatter(x=ticker_df.index, y=ticker_df['close'], mode='lines', name='Close Price'),
#             go.Scatter(x=ticker_df.index, y=ticker_df['short_ma'], mode='lines', name='Short MA'),
#             go.Scatter(x=ticker_df.index, y=ticker_df['long_ma'], mode='lines', name='Long MA')
#         ],
#         'layout': go.Layout(
#             title=f'Stock Prices for {selected_stock}',
#             xaxis={'title': 'Date'},
#             yaxis={'title': 'Price'}
#         )
#     }
#     return figure

# if __name__ == '__main__':
#     app.run_server(debug=True)



In [ ]:
# import plotly.io as pio

# figures_json = {}
# for ticker in tickers:
#     predictions = predictions_dict[ticker]
#     actuals = actuals_dict[ticker]
#     dates = dates_dict[ticker]
    
#     # Retrieve the Mean Squared Error from the dictionary
#     mse = mean_sqr_error[ticker]
    
#     fig = go.Figure()

#     fig.add_trace(go.Scatter(
#         x=dates,
#         y=actuals,
#         mode='lines',
#         name='Actual',
#         line=dict(color='blue')
#     ))

#     fig.add_trace(go.Scatter(
#         x=dates,
#         y=predictions,
#         mode='lines',
#         name=f'Predicted (MSE: {mse:.2f})',
#         line=dict(color='red')
#     ))

#     fig.update_layout(
#         title=f'Actual vs Predicted values for {ticker}',
#         xaxis_title='Date',
#         yaxis_title='Price',
#         legend_title='Legend'
#     )
    
#     figures_json[ticker] = pio.to_json(fig)

# html_content = '''
# <!DOCTYPE html>
# <html>
# <head>
#     <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
# </head>
# <body>
#     <h1>Stock Price Predictions</h1>
#     <label for="ticker-select">Choose a ticker:</label>
#     <select id="ticker-select" onchange="updatePlot()">
#         {options}
#     </select>
#     <div id="plot"></div>
#     <script>
#         var figures = {figures};
        
#         function updatePlot() {{
#             var ticker = document.getElementById('ticker-select').value;
#             var graphDiv = document.getElementById('plot');
#             Plotly.react(graphDiv, JSON.parse(figures[ticker]).data, JSON.parse(figures[ticker]).layout);
#         }}
        
#         // Initial plot
#         document.addEventListener('DOMContentLoaded', function() {{
#             updatePlot();
#         }});
#     </script>
# </body>
# </html>
# '''

# options = ''.join([f'<option value="{ticker}">{ticker}</option>' for ticker in tickers])
# html_content = html_content.format(options=options, figures=figures_json)

# with open('stock_predictions.html', 'w') as f:
#     f.write(html_content)

# print("HTML file with dropdown has been generated.")